<a href="https://colab.research.google.com/github/SanjaySaatyaki/fast_ai_try_out/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.7 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:

#hide
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
!ls {path}

imdb.vocab  README  test  tmp_clas  tmp_lm  train  unsup


In [5]:
files = get_text_files(path, folders=['train','test','unsup'])

In [6]:
txt = files[0].open().read(); txt[:75]

"R.I.C.C.O. is the STUPIDEST film ever made. I can't believe my father bough"

In [7]:
spacy = WordTokenizer()
toks = first(spacy([txt]))

In [8]:
print(coll_repr(toks, 30))

(#151) ['R.I.C.C.O.','is','the','STUPIDEST','film','ever','made','.','I','ca',"n't",'believe','my','father','bought','this','crap','.','This','film','should','da','never','got','made','.','If','this','film','was'...]


In [9]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt),31))

(#155) ['xxbos','xxup','r.i.c.c.o','.','is','the','xxup','stupidest','film','ever','made','.','i','ca',"n't",'believe','my','father','bought','this','crap','.','xxmaj','this','film','should','da','never','got','made','.'...]


In [10]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [11]:
txts = L(o.open().read() for o in files[:2000])

In [18]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return ' '.join(first(sp([txt])))[:400]

In [19]:
subword(10000)

"▁R . I . C . C . O . ▁is ▁the ▁S T U P ID EST ▁film ▁ever ▁made . ▁I ▁can ' t ▁believe ▁my ▁father ▁ bought ▁this ▁crap . ▁This ▁film ▁should ▁da ▁never ▁got ▁made . ▁If ▁this ▁film ▁was ▁wide ▁known , ▁trust ▁me ▁it ▁will ▁be ▁on ▁the ▁#1 ▁spot ▁for ▁IMDB ' s ▁worst . The ▁acting ▁is ▁horrible ▁it ' s ▁scary , which ▁it ▁is ▁why ▁it ' s ▁horror . ▁This ▁piece ▁of ▁ s *** ▁had ▁no ▁horror ▁in ▁at "

In [20]:

toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#155) ['xxbos','xxup','r.i.c.c.o','.','is','the','xxup','stupidest','film','ever','made','.','i','ca',"n't",'believe','my','father','bought','this','crap','.','xxmaj','this','film','should','da','never','got','made','.'...]


In [21]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#155) ['xxbos','xxup','r.i.c.c.o','.','is','the','xxup','stupidest','film','ever','made','.','i','ca',"n't",'believe','my','father','bought','this'...]

In [22]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1920) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','i','in'...]"

In [23]:

nums = num(toks)[:20]; nums

TensorText([   2,    7,    0,   10,   16,    9,    7,    0,   29,  147,   95,   10,   18,  192,   35,  297,   65,  552, 1113,   22])

In [24]:
' '.join(num.vocab[o] for o in nums)

"xxbos xxup xxunk . is the xxup xxunk film ever made . i ca n't believe my father bought this"

In [26]:

#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.
